In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
# from geopandas.tools import sjoin
from shapely.geometry import Point, mapping,shape
import time
from shapely import prepared
import json

In [2]:
zipcodes = gpd.GeoDataFrame.from_file('ZIP_CODE_040114/ZIP_CODE_040114.shp')

In [3]:
zipcodes = zipcodes.to_crs(epsg=4269)

In [4]:
zipcodes = zipcodes[['ZIPCODE','geometry']]
zipcodes.ZIPCODE = zipcodes.ZIPCODE.astype(int)
# zipcodes.rename(columns={'ZIPCODE':'ZIP_CODE'},inplace=True)

In [5]:
filelist = ['2009.csv','2010.csv','2011.csv','2012.csv','2013.csv','2014.csv']
qstr = 'dropoff_latitude > 40.64067 & dropoff_latitude < 40.65138 & dropoff_longitude < -73.776284 & dropoff_longitude > -73.79469'
chunksize = 1000000

In [6]:
iter_csv = pd.read_csv(filelist[5], iterator=True, chunksize=chunksize)
for chunk in iter_csv:
    all_jfk = pd.concat([chunk.query(qstr) for chunk in iter_csv])

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [7]:
spring_jfk = pd.concat([all_jfk[all_jfk['pickup_datetime'].str[5:7] == mm] for mm in ['04','05','06']])
spring_jfk.reset_index(drop=True,inplace=True)
spring_jfk['duration'] = np.divide(pd.to_datetime(spring_jfk['dropoff_datetime']) - pd.to_datetime(spring_jfk['pickup_datetime']),np.timedelta64(1,'s'))
spring_jfk['total_ex_tip'] = spring_jfk['total_amount'] - spring_jfk['tip_amount']
spring_jfk['zipcode'] = ''

In [16]:
print 'Average tip percentage for all trips to JFK (when tip is reported):'
jfk_tiprate = spring_jfk.query('tip_amount > 0')['tip_amount'].sum() / spring_jfk.query('tip_amount > 0')['total_ex_tip'].sum()
print jfk_tiprate

Average tip percentage for all trips to JFK (when tip is reported):
0.189558109134


In [9]:
# jfk = pd.read_pickle('JFKTLC')
# xc = jfk['rate_code'][64:210]
# yc = jfk['store_and_fwd_flag'][64:210]
xc = spring_jfk['pickup_longitude']
yc = spring_jfk['pickup_latitude']

In [10]:
pts = gpd.GeoSeries([Point(x, y) for x, y in zip(xc, yc)])

In [11]:
rides_by_zip = {}
for i in zipcodes.index:
    rides_by_zip[zipcodes.loc[i].ZIPCODE] = pts.loc[pts.intersects(zipcodes.loc[i].geometry)]

In [12]:
zip_centroids = {}
for i in zipcodes.index:
    zip_centroids[zipcodes.loc[i].ZIPCODE] = (yc.iloc[rides_by_zip[zipcodes.loc[i].ZIPCODE].index].mean(),
                                             xc.iloc[rides_by_zip[zipcodes.loc[i].ZIPCODE].index].mean())

In [13]:
with open('JFK2014.json', 'w') as f:
    f.write(json.dumps(zip_centroids))

In [14]:
for k, v in rides_by_zip.iteritems():
    spring_jfk.loc[v.index,'zipcode'] = k

In [35]:
rides_summary = spring_jfk.groupby('zipcode')[['pickup_longitude','pickup_latitude','trip_distance','duration','total_ex_tip']].mean()

In [36]:
rides_summary['total_w_inferred_tip'] = rides_summary['total_ex_tip'] * (1 + jfk_tiprate)

In [37]:
rides_summary.columns = ['taxi_mean_' + col for col in rides_summary.columns]

In [38]:
rides_summary['taxi_N'] = spring_jfk.groupby('zipcode').size()

In [40]:
rides_summary.to_pickle('rides_summary.pkl')

In [20]:
rides_summary = zipcodes.merge(rides_summary,how='left',left_on='ZIPCODE',right_index=True,copy=False)

In [23]:
with open('taxi_by_zip2.geojson', 'w') as f:
    f.write(rides_summary.to_json())

In [22]:
rides_summary.head(20)

,ZIPCODE,geometry,taxi_mean_pickup_longitude,taxi_mean_pickup_latitude,taxi_mean_trip_distance,taxi_mean_duration,taxi_mean_total_ex_tip,taxi_mean_total_w_inferred_tip,taxi_N
0,11436,POLYGON ((-73.80584847647387 40.68290932647581...,-73.799750,40.672203,6.209521,1073.532934,37.845030,45.018862,167
1,11213,"POLYGON ((-73.9373976313981 40.67972958928415,...",-73.938687,40.671567,11.214773,1689.840909,36.670341,43.621501,88
2,11212,POLYGON ((-73.90294132545434 40.67083977593342...,-73.924923,40.662739,10.470000,1551.333333,33.666667,40.048456,3
3,11225,"POLYGON ((-73.9579731604348 40.67065695900899,...",-73.957204,40.663907,12.469444,1938.825397,39.567222,47.067510,126
4,11218,POLYGON ((-73.97208109564257 40.65059658730944...,-73.973293,40.644013,18.017865,2329.438202,53.896067,64.112504,89
5,11226,POLYGON ((-73.96190027968616 40.65487064534704...,-73.959839,40.649368,14.603404,2015.085106,44.471277,52.901168,94
6,11219,POLYGON ((-73.98905871487801 40.64411892873606...,-73.995452,40.638534,21.403750,2527.875000,60.750000,72.265655,8
7,11210,POLYGON ((-73.95839960027369 40.63632662549418...,-73.947000,40.632150,14.010000,1566.111111,41.601667,49.487600,18
8,11230,POLYGON ((-73.96451268730161 40.63668725315803...,-73.965577,40.626661,16.177778,2068.000000,48.577556,57.785825,45
9,11204,POLYGON ((-73.98108387453885 40.63528985595042...,-73.983573,40.615104,18.461250,1973.625000,52.416250,62.352175,8


In [32]:
spring_jfk.query('zipcode == 10304')

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,duration,total_ex_tip,zipcode
130150,CMT,2014-05-26 05:05:02,2014-05-26 05:27:13,2,17,-74.089603,40.608111,2,N,-73.782927,...,CRD,52,0,0.5,3,5.33,60.83,1331,57.83,10304


In [34]:
rides_summary.head()

,ZIPCODE,geometry,taxi_mean_pickup_longitude,taxi_mean_pickup_latitude,taxi_mean_trip_distance,taxi_mean_duration,taxi_mean_total_ex_tip,taxi_mean_total_w_inferred_tip,taxi_N
0,11436,POLYGON ((-73.80584847647387 40.68290932647581...,-73.799750,40.672203,6.209521,1073.532934,37.845030,45.018862,167
1,11213,"POLYGON ((-73.9373976313981 40.67972958928415,...",-73.938687,40.671567,11.214773,1689.840909,36.670341,43.621501,88
2,11212,POLYGON ((-73.90294132545434 40.67083977593342...,-73.924923,40.662739,10.470000,1551.333333,33.666667,40.048456,3
3,11225,"POLYGON ((-73.9579731604348 40.67065695900899,...",-73.957204,40.663907,12.469444,1938.825397,39.567222,47.067510,126
4,11218,POLYGON ((-73.97208109564257 40.65059658730944...,-73.973293,40.644013,18.017865,2329.438202,53.896067,64.112504,89
